In [1]:
!pip install biopython
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.0 MB/s eta 0:00:00


In [2]:
# Refer to the author's code
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [17]:
import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [3]:
# First, download the provided a mystery virus

!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

--2025-06-17 10:55:47--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 99.84.245.194, 99.84.245.120, 99.84.245.66, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|99.84.245.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395781 (387K) [video/m2ts]
Saving to: ‘ads1_week4_reads.fq’

ads1_week4_reads.fq 100%[===================>] 386.50K  --.-KB/s    in 0.1s    

2025-06-17 10:55:47 (3.61 MB/s) - ‘ads1_week4_reads.fq’ saved [395781/395781]



It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings
ABC BCA CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.

What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [4]:
input_strings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
shortest_common_super_string = scs(input_strings)
print(shortest_common_super_string, len(shortest_common_super_string))

CCTTGGATTGC 11


How many different shortest common superstrings are there for the input strings given in the previous question?

In [5]:
def scs_all(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """


    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    shortest_len = len(shortest_sup)

    scs = set()
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if len(sup) <= shortest_len:
            scs.add(sup)
    return scs  # return shortest list

In [6]:
input_strings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
scs_list = scs_all(input_strings)
len(scs_list)

4

In [8]:
# Use SeqIO.parse to handle multiple records in the fastq file
sequences = list(SeqIO.parse("ads1_week4_reads.fq", "fastq"))

# Access the sequence from the first record (if you only need one, or loop through all)
# For this problem, it seems you want to assemble all reads, so we'll get all sequences
reads = [str(seq_record.seq) for seq_record in sequences]

# You can now process the 'reads' list. For example, print the first read and the total number of reads:
print("First read:", reads[0][:20])
print("Number of reads:", len(reads))

# Note: 'mys_virus' variable is not created in this corrected code, as you are working with a list of reads now.
# If you intended to assemble these reads into a single sequence, that would be the next step.

First read: GTCCAGCAGAGCAAGTGATG
Number of reads: 1881


All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

In [11]:
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0

    kmer_dict = {}

    for read in reads:
        for i in range(len(read)-k+1):
            kmer_dict[read[i:i+k]] = set()

    for read in reads:
        for i in range(len(read)-k+1):
            kmer_dict[read[i:i+k]].add(read)

    for read in reads:
        current_kmer_set = kmer_dict[read[-1*k:]]
        for kmer_read in current_kmer_set:
            if read != kmer_read:
                olen = overlap(read, kmer_read, min_length=k)
                if olen > best_olen:
                    reada, readb = read, kmer_read
                    best_olen = olen
    return reada, readb, best_olen

In [12]:
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [13]:
%%time
ss = greedy_scs(reads, 10)

CPU times: user 3min 9s, sys: 1.45 s, total: 3min 11s
Wall time: 3min 13s


In [14]:
len(ss)

15894

In [15]:
i = 0
for s in ss:
    if s == 'A':
        i +=1
print('As: ', i)

As:  4633


How many Ts are there in the full, assembled genome from the previous question?



In [16]:
j = 0
for s in ss:
    if s == 'T':
        j += 1
print('Ts: ', j)

Ts:  3723
